<a href="https://colab.research.google.com/github/kdHyeok/VS-Code_History/blob/master/%EC%BA%A1%EC%8A%A4%ED%86%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import datetime

from datetime import datetime
from urllib.parse import urljoin
import re

def crawl_page(limit, offset):
    url = f"https://www.jj.ac.kr/jj/community/notice01.do?mode=list&&articleLimit={limit}&article.offset={offset}"
    try:
        res = requests.get(url, timeout=5)
        res.encoding = 'utf-8'
        res.raise_for_status()
    except Exception as e:
        print(f"[!] 요청 실패 (offset={offset}) → {e}")
        return []

    soup = bs(res.text, "html.parser")
    box = soup.find("tbody")
    rows = []

    if not box:
        return rows

    for tr in box.find_all("tr"):
        tds = tr.find_all("td")
        if len(tds) < 2:
            continue

        no = tds[0].text.strip()
        content_td = tds[1]

        # 제목
        a_tag = content_td.select_one("div > div > a")
        title = ' '.join(a_tag.stripped_strings) if a_tag else None

        # 링크
        href = a_tag.get("href") if a_tag else None
        link = urljoin("https://www.jj.ac.kr/jj/community/notice01.do", href) if href else None

        # 부서
        dept_span = content_td.select_one("div > div:nth-of-type(3) > span.b-writer")
        dept = dept_span.text.strip() if dept_span else None

        # 게시일
        date_span = content_td.select_one("div > div:nth-of-type(3) > span.b-date")
        date = date_span.text.strip() if date_span else None

        # 조회수
        hit_span = content_td.select_one("div > div:nth-of-type(3) > span.b-hit")
        if hit_span:
            hit_text = hit_span.text.strip()
            match = re.search(r'\d+', hit_text)
            views = int(match.group()) if match else 0
        else:
            views = 0

        # 상단 고정 공지 처리
        if no == "공지":
            if not dept:
                dept = "관리자"
            date = datetime.today().strftime('%Y-%m-%d')
            if views == 0:
                views = 0

        if title:
            rows.append({
                "등록 번호": no,
                "공지 제목": title,
                "부서": dept,
                "등록일": date,
                "조회수": views,
                "링크": link
            })

    return rows


In [ ]:
# 전체 페이지 병렬 크롤링 + 중복 제거
def gongji_crawl_parallel(limit=100, max_pages=80):
    offsets = [page * limit for page in range(max_pages)]
    all_rows = []

    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(lambda offset: crawl_page(limit, offset), offsets))

    for page_rows in results:
        all_rows.extend(page_rows)

    # DataFrame 생성 및 중복 제거
    df = pd.DataFrame(all_rows)
    df.drop_duplicates(subset=["등록 번호", "공지 제목"], inplace=True)

    return df

In [ ]:
pd.set_option('display.max_colwidth', None)
df = gongji_crawl_parallel(limit=100, max_pages=10)

In [ ]:
df.head(50)

,등록 번호,공지 제목,부서,등록일,조회수,링크
0,공지,2025학년도 SUPER 핵심역량 진단 참여 안내,관리자,2025-03-27,0,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478197&article.offset=0&articleLimit=100
1,공지,제13회 미래전북포럼 개최 안내,관리자,2025-03-27,0,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478010&article.offset=0&articleLimit=100
2,공지,2025학년도 1학기 직장(학생)예비군 전입신고서 제출 안내,관리자,2025-03-27,0,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=188625&article.offset=0&articleLimit=100
3,7490,전주대학교 캠퍼스 '낭만 사진 작가' 모집,대외협력홍보실,2025-03-26,194,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478268&article.offset=0&articleLimit=100
4,7489,[교수학습개발센터] 2025-1학기 JJ같이아는 학습동아리,교수학습개발센터,2025-03-26,205,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478257&article.offset=0&articleLimit=100
5,7488,[전북생명의전화] 2025년 대학생 전화상담사 양성교육생 모집,관리자,2025-03-26,126,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478211&article.offset=0&articleLimit=100
6,7487,졸업생 맞춤형 취업지원 프로그램 [졸업생 밸류업 특공대] 신청 모집 안내,대학일자리플러스센터 일,2025-03-25,118,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478205&article.offset=0&articleLimit=100
7,7486,「2025학년도 1학기 JST 공유대학 평생교육 온라인 콘텐츠 운영」 안내,RIS사업단,2025-03-24,141,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478077&article.offset=0&articleLimit=100
8,7485,신축 평화관별관 CCTV 설치에 따른 행정예고,시설지원실,2025-03-23,141,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478069&article.offset=0&articleLimit=100
9,7484,[졸업생 특화] 전주고용센터와 함께하는 성취 프로그램 4월 참여자 모집,대학일자리플러스센터,2025-03-21,444,https://www.jj.ac.kr/jj/community/notice01.do?mode=view&articleNo=478052&article.offset=0&articleLimit=100


# ***크롤링 테스트***

In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

ERROR: Operation cancelled by user
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (238 kB/s)
^C
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 new

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def gongji_print(n, m):
    # 공지사항 URL 구성
    gongji_url = f"https://www.jj.ac.kr/jj/community/notice01.do?mode=list&&articleLimit={n}&article.offset={m}"
    raw = requests.get(gongji_url)
    raw.encoding = 'utf-8'  # 인코딩 명시
    soup = bs(raw.text, "html.parser")

    # 결과 저장용 리스트
    rows = []

    # 공지 테이블 추출
    box = soup.find("tbody")
    for tr in box.find_all("tr"):
        # 등록 번호
        td_list = tr.find_all("td")
        if len(td_list) < 2:
            continue
        no = td_list[0].text.strip()

        # 제목 추출
        a_tag = tr.select_one("td:nth-of-type(2) a")
        if a_tag:
            title = ' '.join(a_tag.stripped_strings)
            rows.append({"등록 번호": no, "공지 제목": title})

    return pd.DataFrame(rows)

# 실행
df = gongji_print(100, 0)
df


,등록 번호,공지 제목
0,공지,2025학년도 SUPER 핵심역량 진단 참여 안내
1,공지,제13회 미래전북포럼 개최 안내
2,공지,2025학년도 1학기 직장(학생)예비군 전입신고서 제출 안내
3,7490,전주대학교 캠퍼스 '낭만 사진 작가' 모집
4,7489,[교수학습개발센터] 2025-1학기 JJ같이아는 학습동아리
...,...,...
98,7395,[산학협력단] 제9회 미래전북포럼 개최 안내
99,7394,[산학혁협력단] 제10회 미래전북포럼(2024 하우와우미래캠프 워크숍) 개최 안내
100,7393,임금협약을 위한 단체교섭 요구사실 공고
101,7392,[원격교육지원센터] 2024년 JJ 공개강좌 페스티벌 안내(경품 증정)


### gpt-test

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 1. 공지사항 페이지 URL
url = "https://www.jj.ac.kr/jj/community/notice01.do"

# 2. 페이지 요청 및 파싱
response = requests.get(url)
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "html.parser")

# 3. 공지사항 제목 추출 (a 태그 중 data-article-no 속성이 있는 것)
notice_links = soup.select('a[data-article-no]')

# 4. 텍스트만 추출해서 리스트로 저장
titles = [link.get_text(strip=True) for link in notice_links]

# 5. pandas DataFrame으로 변환
df = pd.DataFrame(titles, columns=['공지 제목'])

# 6. 출력
df

,공지 제목
0,2025학년도 SUPER 핵심역량 진단 참여 안내
1,제13회 미래전북포럼 개최 안내
2,2025학년도 1학기 직장(학생)예비군 전입신고서 제출 안내
3,전주대학교 캠퍼스 '낭만 사진 작가' 모집
4,[교수학습개발센터] 2025-1학기 JJ같이아는 학습동아리
5,[전북생명의전화] 2025년 대학생 전화상담사 양성교육생 모집
6,졸업생 맞춤형 취업지원 프로그램 [졸업생 밸류업 특공대] 신청 모집 안내
7,「2025학년도 1학기 JST 공유대학 평생교육 온라인 콘텐츠 운영」 안내
8,신축 평화관별관 CCTV 설치에 따른 행정예고
9,[졸업생 특화] 전주고용센터와 함께하는 성취 프로그램 4월 참여자 모집
